In [1]:
# Full Optimized Code for Bird Classification
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress AVX warnings
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'  # Mitigate OpenMP conflict

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np

In [10]:
# Enhanced Data Augmentation with Mixup and CutMix
class AugmentedGenerator(tf.keras.utils.Sequence):
    def __init__(self, generator, alpha=0.4, cutmix_prob=0.5):
        self.generator = generator
        self.alpha = alpha
        self.cutmix_prob = cutmix_prob

    def __len__(self):
        return len(self.generator)

    def __getitem__(self, index):
        images, labels = self.generator[index]
        
        if np.random.rand() < self.cutmix_prob:
            return self.cutmix(images, labels)
        else:
            return self.mixup(images, labels)
    
    def mixup(self, images, labels):
        indices = np.random.permutation(images.shape[0])
        shuffled_images = images[indices]
        shuffled_labels = labels[indices]
        lam = np.random.beta(self.alpha, self.alpha)
        mixed_images = lam * images + (1 - lam) * shuffled_images
        mixed_labels = lam * labels + (1 - lam) * shuffled_labels
        return mixed_images, mixed_labels

    def cutmix(self, images, labels):
        indices = np.random.permutation(images.shape[0])
        shuffled_images = images[indices]
        shuffled_labels = labels[indices]
        lam = np.random.beta(self.alpha, self.alpha)
        bbx1, bby1, bbx2, bby2 = self.rand_bbox(images.shape, lam)
        images[:, bbx1:bbx2, bby1:bby2, :] = shuffled_images[:, bbx1:bbx2, bby1:bby2, :]
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (images.shape[1] * images.shape[2]))
        labels = lam * labels + (1 - lam) * shuffled_labels
        return images, labels

    def rand_bbox(self, size, lam):
        W = size[1]
        H = size[2]
        cut_rat = np.sqrt(1. - lam)
        cut_w = int(W * cut_rat)
        cut_h = int(H * cut_rat)
        cx = np.random.randint(W)
        cy = np.random.randint(H)
        bbx1 = np.clip(cx - cut_w // 2, 0, W)
        bby1 = np.clip(cy - cut_h // 2, 0, H)
        bbx2 = np.clip(cx + cut_w // 2, 0, W)
        bby2 = np.clip(cy + cut_h // 2, 0, H)
        return bbx1, bby1, bbx2, bby2


In [11]:
# Dataset paths (UPDATE THESE TO YOUR LOCAL PATHS)
train_path = 'D:/Dataset/Birds_CNN/Train'
test_path = 'D:/Dataset/Birds_CNN/Test'

In [12]:

# Data Generators with Validation Split
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input,
    rotation_range=60,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.4, 1.6],
    validation_split=0.2  # 80% training, 20% validation
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(480, 480),
    batch_size=4,  # Reduced batch size for memory stability
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(480, 480),
    batch_size=4,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input
)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(480, 480),
    batch_size=4,
    class_mode='categorical',
    shuffle=False
)


Found 91 images belonging to 13 classes.
Found 13 images belonging to 13 classes.
Found 26 images belonging to 13 classes.


In [13]:
# Build Enhanced Model Architecture
base_model = EfficientNetV2L(
    include_top=False,
    weights='imagenet',
    input_shape=(480, 480, 3)
)
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(2048, activation='swish', kernel_regularizer='l2'),
    Dropout(0.5),
    Dense(1024, activation='swish', kernel_regularizer='l2'),
    Dropout(0.4),
    Dense(13, activation='softmax')  # Ensure this matches your class count
])

In [14]:
# COMPILE THE MODEL BEFORE TRAINING
optimizer = Adam(learning_rate=1e-4, amsgrad=True)
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)


In [ ]:
# First Training Phase (Frozen Backbone)
history = model.fit(
    AugmentedGenerator(train_generator),
    validation_data=val_generator,
    epochs=50
)


Epoch 1/50
14/23 [=================>............] - ETA: 33s - loss: 32.0159 - accuracy: 0.0909  

In [ ]:
model.save('D:/Model_Main/Xception_net_client_Snake_cnn.h5')

In [41]:
# Fine-tuning Phase
base_model.trainable = True
for layer in base_model.layers[:300]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

history_fine = model.fit(
    AugmentedGenerator(train_generator),
    validation_data=val_generator,
    initial_epoch=history.epoch[-1],
    epochs=history.epoch[-1] + 30,
    callbacks=[checkpoint, early_stop, reduce_lr]
)

Epoch 50/79
23/23 [==============================] - 276s 10s/step - loss: 12.2241 - accuracy: 0.8242 - val_loss: 11.4779 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 51/79
23/23 [==============================] - 230s 10s/step - loss: 12.2709 - accuracy: 0.8242 - val_loss: 11.5094 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 52/79
23/23 [==============================] - 231s 10s/step - loss: 12.0690 - accuracy: 0.7802 - val_loss: 11.4916 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 53/79
23/23 [==============================] - 230s 10s/step - loss: 12.1788 - accuracy: 0.8462 - val_loss: 11.4097 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 54/79
23/23 [==============================] - 234s 10s/step - loss: 12.0448 - accuracy: 0.8462 - val_loss: 11.3442 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 55/79
23/23 [==============================] - 228s 10s/step - loss: 11.9122 - accuracy: 0.8571 - val_loss: 11.3273 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 56/79
23/23 [=========

In [2]:
model.save('D:/Model_Main/Xception_net_client_bird_cnn.h5')

NameError: name 'model' is not defined

In [6]:
from tensorflow.keras.models import load_model

# Path to your saved model
model_path = 'D:/Model_Main/Xception_net_client_bird_cnn.h5'

# Load the model
model = load_model(model_path)

# Optional: Print model summary to confirm it's loaded correctly
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (Functiona  (None, 15, 15, 1280)     117746848 
 l)                                                              
                                                                 
 global_average_pooling2d_5   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 batch_normalization_5 (Batc  (None, 1280)             5120      
 hNormalization)                                                 
                                                                 
 dense_15 (Dense)            (None, 2048)              2623488   
                                                                 
 dropout_10 (Dropout)        (None, 2048)              0         
                                                      

In [11]:
# Evaluate the model
eval_results = model.evaluate(test_generator)
print(f"Test Loss: {eval_results[0]:.4f} | Test Accuracy: {eval_results[1]:.4f}")

7/7 [==============================] - 33s 3s/step - loss: 10.3675 - accuracy: 1.0000
Test Loss: 10.3675 | Test Accuracy: 1.0000


In [ ]:
# Define class labels manually
from tensorflow.keras.utils import load_img, img_to_array

class_labels = {
    0: 'Bulbul',
    1: 'Cattle',
    2: 'Gracula',
    3: 'Hawk Owl',
    4: 'Magpie Robin',
    5: 'Moorhen',
    6: 'Parrot',
    7: 'Red Dove',
    8: 'Serpent Egal',
    9: 'Teal',
    10: 'Tyto Owl',
    11: 'water hen',
    12: 'Wood pigeon'
}

# Load the trained model
from tensorflow.keras.models import load_model
import numpy as np
from keras.preprocessing import image

# model = load_model('D:/Model_Main/Xception_net_01.h5')  # Update with your saved model path

# Load and preprocess the image
image_path = 'D:/Test_Val/Client_test/l2.JPG'  # Update with your image path
# Load and preprocess the image
test_image = load_img(image_path, target_size=(299, 299))  # Corrected function
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image / 255.0  # Normalize the image


# Predict the class
result = model.predict(test_image)
predicted_class = np.argmax(result)

# Get the predicted label
prediction = class_labels[predicted_class]

print(f'The predicted class is: {prediction}')

In [12]:
# Define class labels manually
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
import numpy as np
from tensorflow.keras.models import load_model

class_labels = {
    0: 'Bulbul',
    1: 'Cattle',
    2: 'Gracula',
    3: 'Hawk Owl',
    4: 'Magpie Robin',
    5: 'Moorhen',
    6: 'Parrot',
    7: 'Red Dove',
    8: 'Serpent Egal',
    9: 'Teal',
    10: 'Tyto Owl',
    11: 'water hen',
    12: 'Wood pigeon'
}

# Load the trained model
# model = load_model('D:/Model_Main/Xception_net_client_bird_cnn.h5')

# Load and preprocess the image
image_path = 'D:/Dataset/Birds_CNN/Train/Bulbul/bulbul3.png'
test_image = load_img(image_path, target_size=(480, 480))  # Match model input size
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = preprocess_input(test_image)  # Use EfficientNetV2 preprocessing

# Predict the class
result = model.predict(test_image)
predicted_class = np.argmax(result)

# Get the predicted label
prediction = class_labels[predicted_class]

print(f'The predicted class is: {prediction}')

1/1 [==============================] - 8s 8s/step
The predicted class is: Bulbul


In [18]:
class_labels = {
    0: 'Bulbul',
    1: 'Cattle',
    2: 'Gracula',
    3: 'Hawk Owl',
    4: 'Magpie Robin',
    5: 'Moorhen',
    6: 'Parrot',
    7: 'Red Dove',
    8: 'Serpent Egal',
    9: 'Teal',
    10: 'Tyto Owl',
    11: 'water hen',
    12: 'Wood pigeon'
}

# Load the trained model
# model = load_model('D:/Model_Main/Xception_net_client_bird_cnn.h5')

# Load and preprocess the image
image_path = 'D:/Bird_cnn_test_out/wh1.jpg'
test_image = load_img(image_path, target_size=(480, 480))  # Match model input size
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = preprocess_input(test_image)  # Use EfficientNetV2 preprocessing

# Predict the class
result = model.predict(test_image)
predicted_class = np.argmax(result)

# Get the predicted label
prediction = class_labels[predicted_class]

print(f'The predicted class is: {prediction}')

1/1 [==============================] - 1s 1s/step
The predicted class is: Moorhen
